# 加载 GRIB 2 消息

nwpc-data 封装 eccodes-python 的 API 接口，提供加载要素场消息（eccodes 中的 GRIB 2 handler）的便捷方法。

## 加载单个要素场

`load_message_from_file` 加载符合条件的**第一个**要素场。

In [1]:
from nwpc_data.grib.eccodes import load_message_from_file

使用 `find_local_file` 获取 GRAPES MESO 3KM 2020 年 4 月 19 日 00 时次 024 时效的 GRIB 2 文件路径

In [14]:
from nwpc_data.data_finder import find_local_file

data_path = find_local_file(
    "grapes_meso_3km/grib2/orig",
    start_time="2020041900",
    forecast_time="024h",
)
data_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_MESO_3KM/Prod-grib/2020041900/ORIG/rmf.hgra.2020041900024.grb2')

加载 850hPa 温度场，返回 eccodes-python 的 GRIB 2 消息。

**注意**：需要使用 `eccodes.codes_release()` 手动释放消息。

In [15]:
t850 = load_message_from_file(
    data_path,
    parameter="t",
    level_type="pl",
    level=850,
)
t850

94251309221936

`t850` 是 eccodes-python 的 GRIB 2 handler，可以使用 eccodes-python 的 API 接口获取 GRIB 2 消息的内容。

使用 eccodes-python 的 `codes_get` 获取要素场的属性信息。

In [19]:
print("shortName:   ", eccodes.codes_get(t850, "shortName"))
print("typeOfLevel: ", eccodes.codes_get(t850, "typeOfLevel"))
print("level:       ", eccodes.codes_get(t850, "level"))

shortName:    t
typeOfLevel:  isobaricInhPa
level:        850


使用 eccodes-python 的 `codes_get_double_array` 接口获取要素场数组。

In [4]:
import eccodes

values = eccodes.codes_get_double_array(t850, "values")
values = values.reshape([1671, 2501])
values

array([[269.6815, 269.6895, 269.6985, ..., 253.6865, 253.6905, 253.6965],
       [269.6965, 269.7045, 269.7225, ..., 253.6575, 253.6945, 253.7115],
       [269.7135, 269.7295, 269.7495, ..., 253.7195, 253.6975, 253.7155],
       ...,
       [294.1695, 294.0615, 293.9815, ..., 291.3685, 291.3925, 291.4115],
       [294.1215, 294.0555, 294.0105, ..., 291.4275, 291.4255, 291.4345],
       [294.0735, 294.0595, 294.0345, ..., 291.5105, 291.4825, 291.4545]])

释放要素场。

In [5]:
eccodes.codes_release(t850)

加载 2m 温度场。

In [6]:
t2m = load_message_from_file(
    data_path,
    parameter="2t",
    level_type="heightAboveGround",
    level=2
)
t2m

94251309199808

In [7]:
eccodes.codes_release(t2m)

## 从单个文件中加载多个要素场

加载符合条件的所有要素场。

In [8]:
from nwpc_data.grib.eccodes import load_messages_from_file

加载所有等压面层的位势高度场，返回 GRIB 2 消息列表。

**注意**：需要使用 `eccodes.codes_release()` 手动释放所有消息。

In [9]:
gh = load_messages_from_file(
    data_path,
    parameter="gh",
    level_type="pl"
)
print(len(gh))

19


显示每个要素场的层次信息。

In [12]:
for message in gh:
    print(eccodes.codes_get(message, "typeOfLevel"), eccodes.codes_get(message, "level"))

isobaricInhPa 1000
isobaricInhPa 975
isobaricInhPa 950
isobaricInhPa 925
isobaricInhPa 900
isobaricInhPa 850
isobaricInhPa 800
isobaricInhPa 750
isobaricInhPa 700
isobaricInhPa 650
isobaricInhPa 600
isobaricInhPa 550
isobaricInhPa 500
isobaricInhPa 450
isobaricInhPa 400
isobaricInhPa 350
isobaricInhPa 300
isobaricInhPa 200
isobaricInhPa 100


释放所有消息

In [13]:
for message in gh:
    eccodes.codes_release(message)

## 下一步

[加载 GRIB 2 要素场](./02.2_field.ipynb) 介绍另一种检索方法，返回 `xarray.DataArray` 的函数。

浏览下面的 notebook，获取关于参数的更多信息：

[parameter](./03.1_parameter.ipynb) 介绍如何设置变量。

[level](./03.2_level.ipynb) 介绍如何设置层次。